Group Name: AG 34.

Student Name (Student ID):

1. Liu yuxin (A0229166Y)

2. Lu lifu  (xxxxxxx)

3. Qin qi  (xxxxxxx)

# Question 1

Consider the maze shown below. The Maze has 16 rows and 24 columns The objective is to find a shortest path from cell $S$ to cell $G$.


![Maze](Maze_Assignment_1.jpg)


The agent can take four actions in each cell: 'RIGHT', 'DOWN', 'UP', 'LEFT'.  

Each cell is represented as $(x,y)$, where $x$ indicates row number and $y$ indicates column number. Action 'UP' takes the agent from cell $(x,y)$ to $(x+1,y)$. Action 'DOWN' takes the agent from cell $(x,y)$ to $(x-1,y)$. Action 'RIGHT' takes the agent from cell $(x,y)$ to $(x,y+1)$. Action 'LEFT' takes the agent from cell $(x,y)$ to $(x,y-1)$. The triplet $(s,a,s')$  indicates that taking action $a$ at state $s$ leads to state $s'$. Actions 'LEFT' or 'RIGHT' cost 10 units for all $(s,a,s')$. Actions 'UP' or 'DOWN' cost 1 unit for all  $(s,a,s')$.  The agent cannot move into cells that are shaded. Assume that the agent knows the boundaries of the maze and has full observability. Consequently, at the bottom (row 0) and top (row 15), the agent will not take actions 'DOWN' and 'UP', respectively; at left (column 0) and right (column 23) columns, the agent will not take 'LEFT' and 'RIGHT' actions, respectively. Similalry, the agent will not take actions that lead to shaded region in the maze.

## **Q1.a: Class Maze(Problem)** [3 Marks]

Write a Maze class to create a model for this problem. You should not use an explicit state space model. The modelling should inherit the abstract class 'Problem' (given below). With the problem formulation, find the shortest path from S to G cell. Propose and implement multiple heuristics (at least two heuristics) for informed search algorithms. 

## **Q1.b: Analysis of the Algorithms** [7 Marks]

1. Solve the above Maze problem using the following algorithms

    a. Breadth-First Search

    b. Depth-First Search with Cycle-Check

    c. Iterative-Deepening Search with Cycle-Check

    d. Uniform-Cost Search

    e. A* Search 

    f. Greedy Best-first Search

    g. Any other variants for search algorithms that are not discussed in the class (bonus/optional question) 

2. Identify the number of nodes generated, number of nodes expanded, maximum frontier size, and path-cost for the above algorithms. 
 
3. Compare the performance of informed search algorithms with proposed heuristics. Identify the best performing heuristic and explain.
 
4. Draw a bar plot comparing the statistics of the algorithms and explain the results. 

Note 1: You must follow the problem formulation discussed in the class. A abstract class for Problem and Node definition is presented below. The search tree generation should follow the template discussed in the class (i.e., Node class, expand methods, etc.). 

Note 2: If you are borrowing a block of code (for example, helper functions or data structures, etc.) from AIMA4e repository, you have to acknowledge it in the code. 

Note 3: The code should be written in a single jupyter notebook file.

In [110]:
class Problem:
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

In [111]:
# Use the following Node class to generate search tree
import math
class Node:
    "A Node in a search tree."
    # add more proprity -> node depth to record how far the node from origin
    def __init__(self, state, parent=None, action=None, path_cost=0,node_depth=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost,node_depth=node_depth)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost 


In [122]:
from unittest import result

class Maze(Problem):
    #Your code goes here

    def __init__(self, initial=None, goal=None, **kwds):
        super().__init__(initial, goal, **kwds)
        self.initial = initial  # store initial state
        self.cur_state = initial
        self.goal = goal  # store goal state
        self.layout = kwds['layout']
        self.Y_of_layout = len(self.layout[0])
        self.X_of_layout = len(self.layout)

    def actions(self,state):
        # return valid actions from this state
        # check four directions RIGHT', 'DOWN', 'UP', 'LEFT'
        valid_actions = []
        x,y = state
        # RIGHT  (x+1,y)
        if  not self.reachBoundary((x+1,y)):
            if not self.isWall((x+1,y)) :
                valid_actions.append('R')
        # LEFT (x-1,y)
        if not self.reachBoundary((x-1,y)):
            if not self.isWall((x-1,y)) :
                valid_actions.append('L')
        # UP (x,y+1)
        if not self.reachBoundary((x,y+1)):
            if not self.isWall((x,y+1)):
                valid_actions.append('U')
        # DOWN (x,y-1)
        if not self.reachBoundary((x,y-1)):
            if not self.isWall((x,y-1)):
                valid_actions.append('D')
        # print(x,y,valid_actions)
        return valid_actions
    
    def result(self, state, action):
        if action == 'R':
            cur_state = (state[0]+1,state[1])
        elif action =='L':
            cur_state = (state[0]-1,state[1])
        elif action =='U':
            cur_state = (state[0],state[1]+1)
        else:
            cur_state = (state[0],state[1]-1)
        return  cur_state

    def action_cost(self, s, a, s1): 
        # Right and Left cost 10
        if a =='R' or a=='L':
            return 10
        return 1
    
    def isWall(self,state):
        # check if state is wall
        x,y = state
        return self.layout[x][y]==1
    
    def reachBoundary(self,state):
        x,y = state
        if x <0 or x >= self.X_of_layout or y<0 or y>=self.Y_of_layout:
            return True
        return False

    def print_layout(self):
        print('X_of_layout: ',self.X_of_layout)
        print('Y_of_layout: ',self.Y_of_layout)
        print('position (x,y) follows the following axis:')
        a = """
Y 
^
|
|
|
|
O —— —— —— ——>X
"""
        print(a) 
        for y in range(self.Y_of_layout-1,-1,-1):
            for x in range(self.X_of_layout):
                print(self.layout[x][y],end=" ")
            print()  

    def test(self):
        print('--------test self.isWall ---------------')
        print(self.isWall(self.goal)==False)
        print(self.isWall(self.initial)==False)
        print(self.isWall((0,0))==False)
        print(self.isWall((0,15))==False)
        print(self.isWall((23,15))==False)
        print(self.isWall((23,0))==False)
        print(self.isWall((9,6))==True)
        print(self.isWall((10,10))==True)

        print('--------test self.reachBoundary ---------------')
        print(self.reachBoundary(self.goal)==False)
        print(self.reachBoundary(self.initial)==False)
        print(self.reachBoundary((0,0))==False)
        print(self.reachBoundary((0,15))==False)
        print(self.reachBoundary((23,15))==False)
        print(self.reachBoundary((23,0))==False)
        print(self.reachBoundary((-1,0))==True)
        print(self.reachBoundary((0,16))==True)

        print('----------test self.actions-------------')
        print(self.actions(self.initial)==['R','U','D'])
        print(self.actions(self.goal)==['L','U'])
        print(self.actions((0,0)) ==['R','U'])
        print(self.actions((0,15)) ==['R','D'])
        print(self.actions((23,15)) ==['L','D'])
        print(self.actions((23,0)) ==['L','U'])
        print(self.actions((9,5)) ==['R','L','D'])
        print(self.actions((11,10))==['U','D'])
        print(self.actions((11,8))==['R','L','U','D'])

        print('----------test self.action_cost -------------')
        print(self.action_cost(self.initial,'R', (self.initial[0]+1, self.initial[1]))==10 )
        print(self.action_cost(self.initial,'D', (self.initial[0], self.initial[1]-1))==1 )
        print(self.action_cost(self.goal,'L', (self.goal[0]-1, self.goal[1]))==10 )
        print(self.action_cost(self.goal,'U', (self.goal[0], self.goal[1]+1))==1 )

        print('----------test self.result -------------')
        print(self.result(self.initial,'R')==(self.initial[0]+1, self.initial[1]) )
        print(self.result(self.initial,'D')==(self.initial[0], self.initial[1]-1) )
        print(self.result(self.goal,'L')==(self.goal[0]-1, self.goal[1]) )
        print(self.result(self.goal,'U')==(self.goal[0], self.goal[1]+1) )
        print('----------test Is_cycle -------------')
        node0 = Node(state=(0,0), parent=None, action=None, path_cost=0)
        node1 = Node(state=(0,1), parent=node0, action=None, path_cost=0)
        print(self.IsCycle(node1) == False)
        node0 = Node(state=(0,0), parent=None, action=None, path_cost=0)
        node1 = Node(state=(0,1), parent=node0, action=None, path_cost=0)
        node2 = Node(state=(0,0), parent=node1, action=None, path_cost=0)
        print(self.IsCycle(node2) == True)       
        node0 = Node(state=(1,1), parent=None, action=None, path_cost=0)
        node1 = Node(state=(1,0), parent=node0, action='D', path_cost=1)
        node2 = Node(state=(2,0), parent=node1, action='R', path_cost=2)
        node3 = Node(state=(2,1), parent=node2, action='U', path_cost=3)
        node4 = Node(state=(1,1), parent=node3, action='L', path_cost=4)
        print(self.IsCycle(node4) == True)   
        print('----------test heuristic1 -------------')
        node0 = Node(state=self.goal)
        node1 = Node(state=self.initial)
        node2 = Node(state=(6,7))
        print(self.heuristic1(node0)==0)
        print(self.heuristic1(node1)==5)
        print(self.heuristic1(node2)==7)
        print('----------test heuristic2 -------------')
        print(self.heuristic2(node0)==0)
        print(self.heuristic2(node1)==13)
        print(self.heuristic2(node2)==34)
        print('----------test heuristic3 -------------')
        print(self.heuristic3(node0)==0)
        print(self.heuristic3(node1)==math.sqrt(10))
        print(self.heuristic3(node2)==5)

    def get_total_path(self,node):
        path = []
        temp = node
        while True:
            path.append( (temp,temp.action, temp.path_cost))
            temp = temp.parent
            if temp is None:
                break
        path = path[::-1]
        return path
    
    def IsCycle(self,node):
        cur_state = node.state
        temp = node.parent
        while True:
            if temp is None:
                return False
            if temp.state == cur_state:
                return True
            temp = temp.parent
        return False
         
    # ---------------------different algotithms -------------------------------

    # ------------------------- bfs ------------------------------------------
    def bfs(self):
        # generated means-> every time we append a node into reached we should number_of_nodes_generated +1
        number_of_nodes_generated = 0
        # expended means-> we already pop this node from frontier. we should number_of_nodes_expanded+1
        number_of_nodes_expanded = 0
        # using max(maximum_frontier_size, len(frontier))
        maximum_frontier_size = 0
        # using deque as the FIFO queue
        from collections import deque
        frontier = deque([]) # frontier is like  [node0 ,node1...],
        node = Node(state=self.initial, parent=None, action=None,path_cost=0)   
        if self.is_goal(node.state):
            return node,0,0,0,0
        frontier.append(node)
        maximum_frontier_size = max(maximum_frontier_size, len(frontier))  # update  maximum_frontier_size 
        reached = {node.state}  # contain { state1,state2...} eg {(0,1), (0,2)...}
        number_of_nodes_expanded += 1
        while frontier != []:
            node = frontier.popleft()  # FIFO queue  popleft!
            number_of_nodes_expanded += 1  # add 1 to node expended
            # get children for this node
            next_actions = self.actions(node.state)
            children = []
            for action in next_actions:
                child_state = self.result(node.state,action)
                action_cost = self.action_cost(node.state,action,child_state)
                child_node = Node(state=child_state, parent=node, action=action,path_cost=node.path_cost+action_cost)
                children.append(child_node)
            
            for child_node in children:
                child_state = child_node.state
                if self.is_goal(child_state):   # reach goal?
                    return child_node,number_of_nodes_generated,number_of_nodes_expanded,maximum_frontier_size, child_node.path_cost
                if child_state not in reached:
                    reached.add(child_state)
                    frontier.append(child_node)
                    number_of_nodes_generated += 1 # this child node is reached so we add 1 in number of node generated
                    maximum_frontier_size = max(maximum_frontier_size, len(frontier))
        return None      

    # ------------------------- dfs -------------------------------------
    def dfs(self):
        # lecture2 62
        number_of_nodes_generated = 0  # related with frontier pop
        number_of_nodes_expanded = 0  # related with reached
        maximum_frontier_size = 0 
        node = Node(state=self.initial, parent=None, action=None,path_cost=0)   
        if self.is_goal(node.state):
            return node,0,0,0,0
        frontier = [] # LIFO frontier is like  [node0 ,node1...],
        frontier.append(node)
        maximum_frontier_size = max(maximum_frontier_size, len(frontier))  # update  maximum_frontier_size 
        reached = {node.state}  # contain { state1,state2...} eg {(0,1), (0,2)...}
        number_of_nodes_expanded += 1
        while frontier != []:
            node = frontier.pop()  # LIFO queue  pop!
            # check if node is cycle, we continue to expand next node
            if self.IsCycle(node):
                continue
            number_of_nodes_expanded += 1  # add 1 to node expended
            # get children for this node
            next_actions = self.actions(node.state)
            children = []
            for action in next_actions:
                child_state = self.result(node.state,action)
                action_cost = self.action_cost(node.state,action,child_state)
                child_node = Node(state=child_state, parent=node, action=action,path_cost=node.path_cost+action_cost)
                children.append(child_node)
            
            for child_node in children:
                child_state = child_node.state
                if self.is_goal(child_state):   # reach goal?
                    return child_node,number_of_nodes_generated,number_of_nodes_expanded,maximum_frontier_size, child_node.path_cost
                if child_state not in reached:
                    reached.add(child_state)
                    frontier.append(child_node)
                    number_of_nodes_generated += 1 # this child node is reached so we add 1 in number of node generated
                    maximum_frontier_size = max(maximum_frontier_size, len(frontier))
        return None        

    # ------------------------- dls -------------------------------------
    def dls(self,depth_limit):
        # lecture2 62
        number_of_nodes_generated = 0  # related with frontier pop
        number_of_nodes_expanded = 0  # related with reached
        maximum_frontier_size = 0 
        node = Node(state=self.initial, parent=None, action=None,path_cost=0,node_depth=0)   
        if self.is_goal(node.state):
            return node,0,0,0,0
        frontier = [] # LIFO frontier is like  [node0 ,node1...],
        frontier.append(node)
        maximum_frontier_size = max(maximum_frontier_size, len(frontier))  # update  maximum_frontier_size 
        reached = {node.state}  # contain { state1,state2...} eg {(0,1), (0,2)...}
        number_of_nodes_expanded += 1
        while frontier != []:
            node = frontier.pop()  # LIFO queue  pop!
            number_of_nodes_expanded += 1  # add 1 to node expended
            if self.is_goal(node.state):   # reach goal?
                return node,number_of_nodes_generated,number_of_nodes_expanded,maximum_frontier_size, node.path_cost       

            if node.node_depth > depth_limit:
                # do not expend this node.
                continue
            else:    
                # check if node is not cycle we do
                if not self.IsCycle(node):
                    # get children for this node
                    next_actions = self.actions(node.state)
                    children = []
                    for action in next_actions:
                        child_state = self.result(node.state,action)
                        action_cost = self.action_cost(node.state,action,child_state)
                        child_node = Node(state=child_state, parent=node, action=action,path_cost=node.path_cost+action_cost,node_depth=node.node_depth+1)
                        children.append(child_node)
                    
                    for child_node in children:
                        child_state = child_node.state
                        if self.is_goal(child_state):   # reach goal?
                            return child_node,number_of_nodes_generated,number_of_nodes_expanded,maximum_frontier_size, child_node.path_cost
                        if child_state not in reached:
                            reached.add(child_state)
                            frontier.append(child_node)
                            number_of_nodes_generated += 1 # this child node is reached so we add 1 in number of node generated
                            maximum_frontier_size = max(maximum_frontier_size, len(frontier))
        return None,number_of_nodes_generated,number_of_nodes_expanded,maximum_frontier_size, 0 

    # ------------------------- ids -------------------------------------
    def ids(self):
        depth_limit = 0
        number_of_nodes_generated = 0
        number_of_nodes_expanded = 0
        maximum_frontier_size = 0
        while True:
            node,nodes_generated,nodes_expanded,frontier_size, path_cost= self.dls(depth_limit=depth_limit)       
            # update node's numbers
            number_of_nodes_generated += nodes_generated
            number_of_nodes_expanded += nodes_expanded
            maximum_frontier_size = max(maximum_frontier_size,frontier_size)
            print('depth ={}  for this round of dls.  nodes_generated = {}, nodes_expanded ={}, frontier_size ={} '.format(depth_limit,nodes_generated,nodes_expanded,frontier_size))
            if node is None:  # which means the goal is not reached we should keep increasing depth_limit
                depth_limit += 1
            else:
                return node,number_of_nodes_generated,number_of_nodes_expanded,maximum_frontier_size, path_cost, depth_limit

    # ------------------------- ucs -------------------------------------------------------------------------
    def ucs(self):
        # lecture2 p34
        # generated means-> every time we append a node into reached we should number_of_nodes_generated +1
        number_of_nodes_generated = 0
        # expended means-> we already pop this node from frontier. we should number_of_nodes_expanded+1
        number_of_nodes_expanded = 0
        # using max(maximum_frontier_size, len(frontier))
        maximum_frontier_size = 0

        import heapq
        frontier = []
        node = Node(state=self.initial, parent=None, action=None,path_cost=0)
        frontier = []  # frontier is like  [(path_cost0, node0) ,(path_cost1,node1)...}, 
        heapq.heappush(frontier,(node.path_cost,node))
        maximum_frontier_size = max(maximum_frontier_size, len(frontier))  # update  maximum_frontier_size 
        reached = {node.state:node}  # contain { state1:node1,state2:node2...} eg {(0,1):node1, (0,2):node2}
        number_of_nodes_expanded += 1
        while frontier != []:
            path_cost, node = heapq.heappop(frontier)
            number_of_nodes_expanded += 1  # add 1 to node expended
            if self.is_goal(node.state):
                return node,number_of_nodes_generated,number_of_nodes_expanded,maximum_frontier_size, node.path_cost
            # get children for this node
            next_actions = self.actions(node.state)
            children = []
            for action in next_actions:
                child_state = self.result(node.state,action)
                action_cost = self.action_cost(node.state,action,child_state)
                child_node = Node(state=child_state, parent=node, action=action,path_cost=node.path_cost+action_cost)
                children.append(child_node)
            
            for child_node in children:
                child_state = child_node.state
                if child_state not in reached or child_node.path_cost < reached[child_state].path_cost:
                    reached[child_state] = child_node
                    heapq.heappush(frontier,(child_node.path_cost, child_node))
                    number_of_nodes_generated += 1 # this child node is reached so we add 1 in number of node generated
            maximum_frontier_size = max(maximum_frontier_size, len(frontier))
        return None
    
    # ---------- heuristic1 using simple manhattan distance
    def heuristic1(self,node):
        # estimate cost from node to goal use manhattan distance
        import math
        value = math.fabs(self.goal[0]-node.state[0])+math.fabs(self.goal[1]-node.state[1])
        return value

    # ---------- heuristic2 using shortest path cost if there was no walls ( weighted manhattan)
    def heuristic2(self,node):
        # estimate cost from node to goal use shortest path
        import math
        value = 10*math.fabs(self.goal[0]-node.state[0])+math.fabs(self.goal[1]-node.state[1])
        return value

    # ---------- heuristic3 using direct distance between source and goal
    def heuristic3(self,node):
        # estimate cost from node to goal use direct path
        import math
        x_dif = self.goal[0]-node.state[0]
        y_dif = self.goal[1]-node.state[1]
        value = math.sqrt(x_dif**2+y_dif**2)
        return value

    # ---------- heuristic4 using 0  for evey node
    def heuristic4(self,node):
        # in this situation. A star becomes ucs
        return 0
    
    # ---------------- A* -----------------------------------------------------
    def A_star(self,heuristic):
        # A star is pretty similar to ucs except we use node.path_cost + h(node) for priority queue 
        # generated means-> every time we append a node into reached we should number_of_nodes_generated +1
        number_of_nodes_generated = 0
        # expended means-> we already pop this node from frontier. we should number_of_nodes_expanded+1
        number_of_nodes_expanded = 0
        # using max(maximum_frontier_size, len(frontier))
        maximum_frontier_size = 0

        import heapq
        frontier = []
        node = Node(state=self.initial, parent=None, action=None,path_cost=0)
        frontier = []  # frontier is like  [(path_cost0, node0) ,(path_cost1,node1)...}, 
        heapq.heappush(frontier,(node.path_cost+heuristic(node),node))
        maximum_frontier_size = max(maximum_frontier_size, len(frontier))  # update  maximum_frontier_size 
        reached = {node.state:node}  # contain { state1:node1,state2:node2...} eg {(0,1):node1, (0,2):node2}
        number_of_nodes_expanded += 1
        while frontier != []:
            path_cost, node = heapq.heappop(frontier)
            number_of_nodes_expanded += 1  # add 1 to node expended
            if self.is_goal(node.state):
                return node,number_of_nodes_generated,number_of_nodes_expanded,maximum_frontier_size, node.path_cost
            # get children for this node
            next_actions = self.actions(node.state)
            children = []
            for action in next_actions:
                child_state = self.result(node.state,action)
                action_cost = self.action_cost(node.state,action,child_state)
                child_node = Node(state=child_state, parent=node, action=action,path_cost=node.path_cost+action_cost)
                children.append(child_node)
            
            for child_node in children:
                child_state = child_node.state
                if child_state not in reached or child_node.path_cost < reached[child_state].path_cost:
                    reached[child_state] = child_node
                    heapq.heappush(frontier,(child_node.path_cost+heuristic(child_node), child_node))
                    number_of_nodes_generated += 1 # this child node is reached so we add 1 in number of node generated
            maximum_frontier_size = max(maximum_frontier_size, len(frontier))
        return None
    
    # ----------------greedy best first search -----------------------------------------------------
    def greedy_best_first_search(self,heuristic):
        # greedy best first search is pretty similar to ucs except we use h(node) for priority queue 
        # generated means-> every time we append a node into reached we should number_of_nodes_generated +1
        number_of_nodes_generated = 0
        # expended means-> we already pop this node from frontier. we should number_of_nodes_expanded+1
        number_of_nodes_expanded = 0
        # using max(maximum_frontier_size, len(frontier))
        maximum_frontier_size = 0

        import heapq
        frontier = []
        node = Node(state=self.initial, parent=None, action=None,path_cost=0)
        frontier = []  # frontier is like  [(path_cost0, node0) ,(path_cost1,node1)...}, 
        heapq.heappush(frontier,(heuristic(node),node))
        maximum_frontier_size = max(maximum_frontier_size, len(frontier))  # update  maximum_frontier_size 
        reached = {node.state:node}  # contain { state1:node1,state2:node2...} eg {(0,1):node1, (0,2):node2}
        number_of_nodes_expanded += 1
        while frontier != []:
            path_cost, node = heapq.heappop(frontier)
            number_of_nodes_expanded += 1  # add 1 to node expended
            if self.is_goal(node.state):
                return node,number_of_nodes_generated,number_of_nodes_expanded,maximum_frontier_size, node.path_cost
            # get children for this node
            next_actions = self.actions(node.state)
            children = []
            for action in next_actions:
                child_state = self.result(node.state,action)
                action_cost = self.action_cost(node.state,action,child_state)
                child_node = Node(state=child_state, parent=node, action=action,path_cost=node.path_cost+action_cost)
                children.append(child_node)
            
            for child_node in children:
                child_state = child_node.state
                if child_state not in reached or child_node.path_cost < reached[child_state].path_cost:
                    reached[child_state] = child_node
                    heapq.heappush(frontier,(heuristic(child_node), child_node))
                    number_of_nodes_generated += 1 # this child node is reached so we add 1 in number of node generated
            maximum_frontier_size = max(maximum_frontier_size, len(frontier))
        return None


In [123]:
# ----------- initialization  ---------------------
initial = (10,8)
goal = (9,11)
# initial maze layout
maze_layout = [[0 for i in range(16)] for j in range(24)]
walls = [(9,6),(9,7),(9,8),(9,9),(9,10),(9,14),(10,10),(10,11),(10,12),(10,13),(12,10),(12,11),(13,10),(13,11)]
for wall in walls:
    x,y = wall
    maze_layout[x][y] = 1

maze  = Maze(initial,goal,layout=maze_layout)

In [124]:
# ------------Breadth-First Search -----------------
node,number_of_nodes_generated,number_of_nodes_expanded,maximum_frontier_size, path_cost= maze.bfs()
path = maze.get_total_path(node)
print('----------results for bfs------------')
print('path is shown below:')
print(path)
print('number_of_nodes_generated:',number_of_nodes_generated)
print('number_of_nodes_expanded:',number_of_nodes_expanded)
print('maximum_frontier_size:',maximum_frontier_size)
print('path cost:',path_cost)

----------results for bfs------------
path is shown below:
[(<(10, 8)>, None, 0), (<(10, 7)>, 'D', 1), (<(10, 6)>, 'D', 2), (<(10, 5)>, 'D', 3), (<(9, 5)>, 'L', 13), (<(8, 5)>, 'L', 23), (<(8, 6)>, 'U', 24), (<(8, 7)>, 'U', 25), (<(8, 8)>, 'U', 26), (<(8, 9)>, 'U', 27), (<(8, 10)>, 'U', 28), (<(8, 11)>, 'U', 29), (<(9, 11)>, 'R', 39)]
number_of_nodes_generated: 205
number_of_nodes_expanded: 177
maximum_frontier_size: 32
path cost: 39


In [125]:
# ------------Depth-First Search with Cycle-Check -----------------
node,number_of_nodes_generated,number_of_nodes_expanded,maximum_frontier_size, path_cost= maze.dfs()
path = maze.get_total_path(node)
print('----------results for dfs------------')
print('we should be aware that the children order in each step will change the results of dfs. Which means self.actions() will influence self.dfs() results')
print('path is shown below:')
print(path)
print('number_of_nodes_generated:',number_of_nodes_generated)
print('number_of_nodes_expanded:',number_of_nodes_expanded)
print('maximum_frontier_size:',maximum_frontier_size)
print('path cost:',path_cost)

----------results for dfs------------
we should be aware that the children order in each step will change the results of dfs. Which means self.actions() will influence self.dfs() results
path is shown below:
[(<(10, 8)>, None, 0), (<(10, 7)>, 'D', 1), (<(10, 6)>, 'D', 2), (<(10, 5)>, 'D', 3), (<(10, 4)>, 'D', 4), (<(10, 3)>, 'D', 5), (<(10, 2)>, 'D', 6), (<(10, 1)>, 'D', 7), (<(10, 0)>, 'D', 8), (<(9, 0)>, 'L', 18), (<(8, 0)>, 'L', 28), (<(8, 1)>, 'U', 29), (<(8, 2)>, 'U', 30), (<(8, 3)>, 'U', 31), (<(8, 4)>, 'U', 32), (<(8, 5)>, 'U', 33), (<(8, 6)>, 'U', 34), (<(8, 7)>, 'U', 35), (<(8, 8)>, 'U', 36), (<(8, 9)>, 'U', 37), (<(8, 10)>, 'U', 38), (<(8, 11)>, 'U', 39), (<(9, 11)>, 'R', 49)]
number_of_nodes_generated: 47
number_of_nodes_expanded: 23
maximum_frontier_size: 27
path cost: 49


Analysis for DFS
When use  ['R','L','U','D'] order in self.actions
we get the following results:

path is shown below:
[(<(10, 8)>, None, 0), (<(10, 7)>, 'D', 1), (<(10, 6)>, 'D', 2), (<(10, 5)>, 'D', 3), (<(10, 4)>, 'D', 4), (<(10, 3)>, 'D', 5), (<(10, 2)>, 'D', 6), (<(10, 1)>, 'D', 7), (<(10, 0)>, 'D', 8), (<(9, 0)>, 'L', 18), (<(8, 0)>, 'L', 28), (<(8, 1)>, 'U', 29), (<(8, 2)>, 'U', 30), (<(8, 3)>, 'U', 31), (<(8, 4)>, 'U', 32), (<(8, 5)>, 'U', 33), (<(8, 6)>, 'U', 34), (<(8, 7)>, 'U', 35), (<(8, 8)>, 'U', 36), (<(8, 9)>, 'U', 37), (<(8, 10)>, 'U', 38), (<(8, 11)>, 'U', 39), (<(9, 11)>, 'R', 49)]
number_of_nodes_generated: 47
number_of_nodes_expanded: 23
maximum_frontier_size: 27
path cost: 49
Since dfs is LIFO and we use ['R','L','U','D'] order. Thus, dfs will first search in 'D' direction.


When use  ['D','U','L','R'] order in self.actions
we get the following results:
path is shown below:
[(<(10, 8)>, None, 0), (<(11, 8)>, 'R', 10), (<(12, 8)>, 'R', 20), (<(13, 8)>, 'R', 30), (<(14, 8)>, 'R', 40), (<(15, 8)>, 'R', 50), (<(16, 8)>, 'R', 60), (<(17, 8)>, 'R', 70), (<(18, 8)>, 'R', 80), (<(19, 8)>, 'R', 90), (<(20, 8)>, 'R', 100), (<(21, 8)>, 'R', 110), (<(22, 8)>, 'R', 120), (<(23, 8)>, 'R', 130), (<(23, 9)>, 'U', 131), (<(23, 10)>, 'U', 132), (<(22, 10)>, 'L', 142), (<(21, 10)>, 'L', 152), (<(20, 10)>, 'L', 162), (<(19, 10)>, 'L', 172), (<(18, 10)>, 'L', 182), (<(17, 10)>, 'L', 192), (<(16, 10)>, 'L', 202), (<(15, 10)>, 'L', 212), (<(14, 10)>, 'L', 222), (<(14, 11)>, 'U', 223), (<(14, 12)>, 'U', 224), (<(15, 12)>, 'R', 234), (<(16, 12)>, 'R', 244), (<(17, 12)>, 'R', 254), (<(18, 12)>, 'R', 264), (<(19, 12)>, 'R', 274), (<(20, 12)>, 'R', 284), (<(21, 12)>, 'R', 294), (<(22, 12)>, 'R', 304), (<(23, 12)>, 'R', 314), (<(23, 13)>, 'U', 315), (<(23, 14)>, 'U', 316), (<(22, 14)>, 'L', 326), (<(21, 14)>, 'L', 336), (<(20, 14)>, 'L', 346), (<(19, 14)>, 'L', 356), (<(18, 14)>, 'L', 366), (<(17, 14)>, 'L', 376), (<(16, 14)>, 'L', 386), (<(15, 14)>, 'L', 396), (<(14, 14)>, 'L', 406), (<(13, 14)>, 'L', 416), (<(12, 14)>, 'L', 426), (<(11, 14)>, 'L', 436), (<(10, 14)>, 'L', 446), (<(10, 15)>, 'U', 447), (<(9, 15)>, 'L', 457), (<(8, 15)>, 'L', 467), (<(7, 15)>, 'L', 477), (<(6, 15)>, 'L', 487), (<(5, 15)>, 'L', 497), (<(4, 15)>, 'L', 507), (<(3, 15)>, 'L', 517), (<(2, 15)>, 'L', 527), (<(1, 15)>, 'L', 537), (<(0, 15)>, 'L', 547), (<(0, 14)>, 'D', 548), (<(0, 13)>, 'D', 549), (<(1, 13)>, 'R', 559), (<(2, 13)>, 'R', 569), (<(3, 13)>, 'R', 579), (<(4, 13)>, 'R', 589), (<(5, 13)>, 'R', 599), (<(6, 13)>, 'R', 609), (<(7, 13)>, 'R', 619), (<(8, 13)>, 'R', 629), (<(9, 13)>, 'R', 639), (<(9, 12)>, 'D', 640), (<(9, 11)>, 'D', 641)]
number_of_nodes_generated: 152
number_of_nodes_expanded: 75
maximum_frontier_size: 80
path cost: 641
Because this time. dfs search towards right, A wrong ditection to goal. Thus the result is very bad.


In [126]:
# ------------Depth-limit Search with Cycle-Check -----------------
# this is to test the correctness of self.dls()
node,number_of_nodes_generated,number_of_nodes_expanded,maximum_frontier_size, path_cost= maze.dls(depth_limit=2)
print(node)


None


In [127]:
# ------------Iterative-Deepening Search with Cycle-Check -----------------
node,number_of_nodes_generated,number_of_nodes_expanded,maximum_frontier_size, path_cost, depth_limit= maze.ids()
print(node)
path = maze.get_total_path(node)
print('----------results for ids------------')
print('path is shown below:')
print(path)
print('number_of_nodes_generated:',number_of_nodes_generated)
print('number_of_nodes_expanded:',number_of_nodes_expanded)
print('maximum_frontier_size:',maximum_frontier_size)
print('path cost:',path_cost)
print('At depth_limit = {} we find the solution'.format(depth_limit))

depth =0  for this round of dls.  nodes_generated = 3, nodes_expanded =5, frontier_size =3 
depth =1  for this round of dls.  nodes_generated = 7, nodes_expanded =9, frontier_size =4 
depth =2  for this round of dls.  nodes_generated = 13, nodes_expanded =15, frontier_size =5 
depth =3  for this round of dls.  nodes_generated = 19, nodes_expanded =21, frontier_size =7 
depth =4  for this round of dls.  nodes_generated = 27, nodes_expanded =29, frontier_size =9 
depth =5  for this round of dls.  nodes_generated = 36, nodes_expanded =38, frontier_size =11 
depth =6  for this round of dls.  nodes_generated = 48, nodes_expanded =50, frontier_size =13 
depth =7  for this round of dls.  nodes_generated = 83, nodes_expanded =85, frontier_size =15 
depth =8  for this round of dls.  nodes_generated = 91, nodes_expanded =93, frontier_size =16 
depth =9  for this round of dls.  nodes_generated = 85, nodes_expanded =87, frontier_size =16 
depth =10  for this round of dls.  nodes_generated = 91, no

In [128]:
# ------------Uniform-Cost Search -----------------
node,number_of_nodes_generated,number_of_nodes_expanded,maximum_frontier_size, path_cost= maze.ucs()
path = maze.get_total_path(node)
print('----------results for ucs------------')
print('path is shown below:')
print(path)
print('number_of_nodes_generated:',number_of_nodes_generated)
print('number_of_nodes_expanded:',number_of_nodes_expanded)
print('maximum_frontier_size:',maximum_frontier_size)
print('path cost:',path_cost)

----------results for ucs------------
path is shown below:
[(<(10, 8)>, None, 0), (<(10, 7)>, 'D', 1), (<(10, 6)>, 'D', 2), (<(10, 5)>, 'D', 3), (<(9, 5)>, 'L', 13), (<(8, 5)>, 'L', 23), (<(8, 6)>, 'U', 24), (<(8, 7)>, 'U', 25), (<(8, 8)>, 'U', 26), (<(8, 9)>, 'U', 27), (<(8, 10)>, 'U', 28), (<(8, 11)>, 'U', 29), (<(9, 11)>, 'R', 39)]
number_of_nodes_generated: 123
number_of_nodes_expanded: 93
maximum_frontier_size: 34
path cost: 39


In [129]:
# -------------------A* Search  with heuristics1 -------------------
node,number_of_nodes_generated,number_of_nodes_expanded,maximum_frontier_size, path_cost= maze.A_star(maze.heuristic1)
path = maze.get_total_path(node)
print('----------results for A* with heuristics1 ------------')
print('path is shown below:')
print(path)
print('number_of_nodes_generated:',number_of_nodes_generated)
print('number_of_nodes_expanded:',number_of_nodes_expanded)
print('maximum_frontier_size:',maximum_frontier_size)
print('path cost:',path_cost)

----------results for A* with heuristics1 ------------
path is shown below:
[(<(10, 8)>, None, 0), (<(10, 7)>, 'D', 1), (<(10, 6)>, 'D', 2), (<(10, 5)>, 'D', 3), (<(9, 5)>, 'L', 13), (<(8, 5)>, 'L', 23), (<(8, 6)>, 'U', 24), (<(8, 7)>, 'U', 25), (<(8, 8)>, 'U', 26), (<(8, 9)>, 'U', 27), (<(8, 10)>, 'U', 28), (<(8, 11)>, 'U', 29), (<(9, 11)>, 'R', 39)]
number_of_nodes_generated: 98
number_of_nodes_expanded: 67
maximum_frontier_size: 34
path cost: 39


In [130]:
# -------------------A* Search  with heuristics2 -------------------
node,number_of_nodes_generated,number_of_nodes_expanded,maximum_frontier_size, path_cost= maze.A_star(maze.heuristic2)
path = maze.get_total_path(node)
print('----------results for A* with heuristics2 ------------')
print('path is shown below:')
print(path)
print('number_of_nodes_generated:',number_of_nodes_generated)
print('number_of_nodes_expanded:',number_of_nodes_expanded)
print('maximum_frontier_size:',maximum_frontier_size)
print('path cost:',path_cost)

----------results for A* with heuristics2 ------------
path is shown below:
[(<(10, 8)>, None, 0), (<(10, 7)>, 'D', 1), (<(10, 6)>, 'D', 2), (<(10, 5)>, 'D', 3), (<(9, 5)>, 'L', 13), (<(8, 5)>, 'L', 23), (<(8, 6)>, 'U', 24), (<(8, 7)>, 'U', 25), (<(8, 8)>, 'U', 26), (<(8, 9)>, 'U', 27), (<(8, 10)>, 'U', 28), (<(8, 11)>, 'U', 29), (<(9, 11)>, 'R', 39)]
number_of_nodes_generated: 62
number_of_nodes_expanded: 36
maximum_frontier_size: 29
path cost: 39


In [131]:
# -------------------A* Search  with heuristics3 -------------------
node,number_of_nodes_generated,number_of_nodes_expanded,maximum_frontier_size, path_cost= maze.A_star(maze.heuristic3)
path = maze.get_total_path(node)
print('----------results for A* with heuristics3 ------------')
print('path is shown below:')
print(path)
print('number_of_nodes_generated:',number_of_nodes_generated)
print('number_of_nodes_expanded:',number_of_nodes_expanded)
print('maximum_frontier_size:',maximum_frontier_size)
print('path cost:',path_cost)

----------results for A* with heuristics3 ------------
path is shown below:
[(<(10, 8)>, None, 0), (<(10, 7)>, 'D', 1), (<(10, 6)>, 'D', 2), (<(10, 5)>, 'D', 3), (<(9, 5)>, 'L', 13), (<(8, 5)>, 'L', 23), (<(8, 6)>, 'U', 24), (<(8, 7)>, 'U', 25), (<(8, 8)>, 'U', 26), (<(8, 9)>, 'U', 27), (<(8, 10)>, 'U', 28), (<(8, 11)>, 'U', 29), (<(9, 11)>, 'R', 39)]
number_of_nodes_generated: 100
number_of_nodes_expanded: 69
maximum_frontier_size: 34
path cost: 39


In [132]:
# -------------------A* Search  with heuristics4 -------------------
node,number_of_nodes_generated,number_of_nodes_expanded,maximum_frontier_size, path_cost= maze.A_star(maze.heuristic4)
path = maze.get_total_path(node)
print('----------results for A* with heuristics4 ------------')
print('path is shown below:')
print(path)
print('number_of_nodes_generated:',number_of_nodes_generated)
print('number_of_nodes_expanded:',number_of_nodes_expanded)
print('maximum_frontier_size:',maximum_frontier_size)
print('path cost:',path_cost)

----------results for A* with heuristics4 ------------
path is shown below:
[(<(10, 8)>, None, 0), (<(10, 7)>, 'D', 1), (<(10, 6)>, 'D', 2), (<(10, 5)>, 'D', 3), (<(9, 5)>, 'L', 13), (<(8, 5)>, 'L', 23), (<(8, 6)>, 'U', 24), (<(8, 7)>, 'U', 25), (<(8, 8)>, 'U', 26), (<(8, 9)>, 'U', 27), (<(8, 10)>, 'U', 28), (<(8, 11)>, 'U', 29), (<(9, 11)>, 'R', 39)]
number_of_nodes_generated: 123
number_of_nodes_expanded: 93
maximum_frontier_size: 34
path cost: 39


In [133]:
#  -----------------Greedy Best-first Search with heuristics1---------------------
node,number_of_nodes_generated,number_of_nodes_expanded,maximum_frontier_size, path_cost= maze.greedy_best_first_search(maze.heuristic1)
path = maze.get_total_path(node)
print('----------results for greedy_best_first_search with heuristics1 ------------')
print('path is shown below:')
print(path)
print('number_of_nodes_generated:',number_of_nodes_generated)
print('number_of_nodes_expanded:',number_of_nodes_expanded)
print('maximum_frontier_size:',maximum_frontier_size)
print('path cost:',path_cost)

----------results for greedy_best_first_search with heuristics1 ------------
path is shown below:
[(<(10, 8)>, None, 0), (<(10, 9)>, 'U', 1), (<(11, 9)>, 'R', 11), (<(11, 10)>, 'U', 12), (<(11, 11)>, 'U', 13), (<(11, 12)>, 'U', 14), (<(11, 13)>, 'U', 15), (<(11, 14)>, 'U', 16), (<(10, 14)>, 'L', 26), (<(10, 15)>, 'U', 27), (<(9, 15)>, 'L', 37), (<(8, 15)>, 'L', 47), (<(8, 14)>, 'D', 48), (<(8, 13)>, 'D', 49), (<(8, 12)>, 'D', 50), (<(8, 11)>, 'D', 51), (<(9, 11)>, 'R', 61)]
number_of_nodes_generated: 38
number_of_nodes_expanded: 24
maximum_frontier_size: 17
path cost: 61


In [134]:
#  -----------------Greedy Best-first Search with heuristics2---------------------
node,number_of_nodes_generated,number_of_nodes_expanded,maximum_frontier_size, path_cost= maze.greedy_best_first_search(maze.heuristic2)
path = maze.get_total_path(node)
print('----------results for greedy_best_first_search with heuristics2 ------------')
print('path is shown below:')
print(path)
print('number_of_nodes_generated:',number_of_nodes_generated)
print('number_of_nodes_expanded:',number_of_nodes_expanded)
print('maximum_frontier_size:',maximum_frontier_size)
print('path cost:',path_cost)

----------results for greedy_best_first_search with heuristics2 ------------
path is shown below:
[(<(10, 8)>, None, 0), (<(10, 7)>, 'D', 1), (<(10, 6)>, 'D', 2), (<(10, 5)>, 'D', 3), (<(9, 5)>, 'L', 13), (<(8, 5)>, 'L', 23), (<(8, 6)>, 'U', 24), (<(8, 7)>, 'U', 25), (<(8, 8)>, 'U', 26), (<(8, 9)>, 'U', 27), (<(8, 10)>, 'U', 28), (<(8, 11)>, 'U', 29), (<(9, 11)>, 'R', 39)]
number_of_nodes_generated: 41
number_of_nodes_expanded: 20
maximum_frontier_size: 24
path cost: 39


In [135]:
#  -----------------Greedy Best-first Search with heuristics3---------------------
node,number_of_nodes_generated,number_of_nodes_expanded,maximum_frontier_size, path_cost= maze.greedy_best_first_search(maze.heuristic3)
path = maze.get_total_path(node)
print('----------results for greedy_best_first_search with heuristics3 ------------')
print('path is shown below:')
print(path)
print('number_of_nodes_generated:',number_of_nodes_generated)
print('number_of_nodes_expanded:',number_of_nodes_expanded)
print('maximum_frontier_size:',maximum_frontier_size)
print('path cost:',path_cost)

----------results for greedy_best_first_search with heuristics3 ------------
path is shown below:
[(<(10, 8)>, None, 0), (<(10, 9)>, 'U', 1), (<(11, 9)>, 'R', 11), (<(11, 10)>, 'U', 12), (<(11, 11)>, 'U', 13), (<(11, 12)>, 'U', 14), (<(11, 13)>, 'U', 15), (<(11, 14)>, 'U', 16), (<(10, 14)>, 'L', 26), (<(10, 15)>, 'U', 27), (<(9, 15)>, 'L', 37), (<(8, 15)>, 'L', 47), (<(8, 14)>, 'D', 48), (<(8, 13)>, 'D', 49), (<(8, 12)>, 'D', 50), (<(8, 11)>, 'D', 51), (<(9, 11)>, 'R', 61)]
number_of_nodes_generated: 38
number_of_nodes_expanded: 24
maximum_frontier_size: 17
path cost: 61


In [136]:
#  -----------------Greedy Best-first Search with heuristics4---------------------
node,number_of_nodes_generated,number_of_nodes_expanded,maximum_frontier_size, path_cost= maze.greedy_best_first_search(maze.heuristic4)
path = maze.get_total_path(node)
print('----------results for greedy_best_first_search with heuristics4 ------------')
print('path is shown below:')
print(path)
print('number_of_nodes_generated:',number_of_nodes_generated)
print('number_of_nodes_expanded:',number_of_nodes_expanded)
print('maximum_frontier_size:',maximum_frontier_size)
print('path cost:',path_cost)

----------results for greedy_best_first_search with heuristics4 ------------
path is shown below:
[(<(10, 8)>, None, 0), (<(10, 7)>, 'D', 1), (<(10, 6)>, 'D', 2), (<(10, 5)>, 'D', 3), (<(9, 5)>, 'L', 13), (<(8, 5)>, 'L', 23), (<(8, 6)>, 'U', 24), (<(8, 7)>, 'U', 25), (<(8, 8)>, 'U', 26), (<(8, 9)>, 'U', 27), (<(8, 10)>, 'U', 28), (<(8, 11)>, 'U', 29), (<(9, 11)>, 'R', 39)]
number_of_nodes_generated: 123
number_of_nodes_expanded: 93
maximum_frontier_size: 34
path cost: 39
